In [1]:
%matplotlib inline
import pandas as pd
import pdb
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from pathlib import Path
from collections import Counter
from util import *
import pickle
import time
import os
import sys

In [ ]:
home = str(Path.home())
DATA_DIR = f'{home}/Downloads/datacenter-challenge'
user_df = pd.read_csv(f'{DATA_DIR}/scheduler_data.csv')
user_df['run_time_from_user_df']=user_df['time_end']-user_df['time_start']

start = time.time()
gpu_ts_main = pd.read_csv(f"{DATA_DIR}/nvidia_smi.csv")
end = time.time()
print (f"Sampling freq : {gpu_ts_main.iloc[1]['timestamp']-gpu_ts_main.iloc[0]['timestamp']}")
# gpu_ts_main.shape #(477606994, 12)
gpu_ts_mini=gpu_ts_main[['Node', 'gpu_index','pcie_link_width_current','timestamp','id_job','utilization_memory_pct','utilization_gpu_pct','power_draw_W','memory_used_MiB']]
start = time.time()
gpu_ts_mini=gpu_ts_mini.set_index('id_job')
end = time.time()
print(end - start)
start = time.time()
hash_ts_length=Counter(gpu_ts_mini.index)
end = time.time()
print(end - start)
print (len(hash_ts_length))

In [ ]:
if (user_df.shape[0]!=len(user_df['id_job'].unique())): #if there are duplicates
    tmp_df=user_df[user_df['id_job'].duplicated(keep=False)][['id_job', 'derived_ec', 'state', 'derived_es']] #may need to rewrite for 2021-IAP. it doesnt have these columns
    idx_to_drop=tmp_df.index[tmp_df['state']!=3]
    user_df_no_dup=user_df.drop(idx_to_drop)
    assert (user_df_no_dup.shape[0]==len(user_df_no_dup['id_job'].unique())) #all unique now
else:
    user_df_no_dup=user_df
set_ts=set(gpu_ts_mini.index)

start = time.time()
jobs_to_drop=[job for job in hash_ts_length if hash_ts_length[job]<300] #drop small length jobs. sample freq seems 100ms
end = time.time()
print(end - start)
print (len(jobs_to_drop))

start = time.time()
gpu_ts=gpu_ts_mini.drop(pd.Index(jobs_to_drop))
end = time.time()
print(end - start)

hash_ts_length=Counter(gpu_ts.index)
len(hash_ts_length)
start = time.time()

gpu_ts=gpu_ts.dropna() #removes NaN timestamps if any

hash_ts_length=Counter(gpu_ts.index)
len(hash_ts_length)

gpu_ts_rst_idx=gpu_ts.reset_index()

In [ ]:
dict_jobs_to_rle_sm=dict()
dict_jobs_to_rle_sm_active=dict()
dict_jobs_to_rle_sm_all=dict()

jobs_set=list(set(gpu_ts.index))

start=time.time()
for job in jobs_set:
    series1=gpu_ts.loc[job]['utilization_gpu_pct']
    dict_jobs_to_rle_sm_active[job]=find_active(series1)
end=time.time()


start=time.time()
for job in jobs_set:
    series1=gpu_ts.loc[job]['utilization_gpu_pct']
    dict_jobs_to_rle_sm_all[job]=find_all(series1)
end=time.time()

start=time.time()
for job in jobs_set:
    series1=gpu_ts.loc[job]['utilization_gpu_pct']
    dict_jobs_to_rle_sm[job]=find(series1)
end=time.time()
end-start

_object = dict_jobs_to_rle_sm_all
_file = open('dict_jobs_to_rle_sm_all.obj', 'wb')
pickle.dump(_object, _file) #https://www.thoughtco.com/using-pickle-to-save-objects-2813661
_file.close()

_object = dict_jobs_to_rle_sm_active
_file = open('dict_active.obj', 'wb')
pickle.dump(_object, _file) #https://www.thoughtco.com/using-pickle-to-save-objects-2813661
_file.close()


_object = dict_jobs_to_rle_sm
_file = open('dict.obj', 'wb')
pickle.dump(_object, _file) #https://www.thoughtco.com/using-pickle-to-save-objects-2813661
_file.close()


_file = open('dict_active.obj', 'rb')
dict_jobs_to_rle_sm_active=pickle.load(_file) #https://www.thoughtco.com/using-pickle-to-save-objects-2813661
_file.close()


_file = open('dict.obj', 'rb')
dict_jobs_to_rle_sm=pickle.load(_file) #https://www.thoughtco.com/using-pickle-to-save-objects-2813661
_file.close()

df=pd.DataFrame()
df['Idle_steps']=pd.Series(dtype=object)
df['Active_steps_list']=pd.Series(dtype=object)
df['All_steps_list']=pd.Series(dtype=object)
df['job_id']=(dict_jobs_to_rle_sm.keys())
df=df.set_index('job_id')
for job in df.index:
    df.loc[job,'Idle_steps']=dict_jobs_to_rle_sm[job]
    df.loc[job,'Active_steps_list']=dict_jobs_to_rle_sm_active[job]
    df.loc[job,'All_steps_list']=dict_jobs_to_rle_sm_all[job]

df_ck1=df

df_ck1['t_list']=None

In [ ]:
df_ck1['total_idle_steps']=df_ck1['Idle_steps'].apply(lambda x: sum(x))
df_ck1['total_active_steps_sanity']=df_ck1['Active_steps_list'].apply(lambda x: sum(x))
for job in df_ck1.index:
    df_ck1.loc[job,'total_steps']=hash_ts_length[job]
df_ck1['Idle_steps_rm1']=df_ck1['Idle_steps'].apply(lambda x: x[1:])
df_ck1['count_intervals_rm1']=df_ck1['Idle_steps_rm1'].apply(lambda x: len(x))
df_ck1['idle_time_pct']=df_ck1['total_idle_steps']/df_ck1['total_steps']*100

df_ck1['Active_Steps']=df_ck1['total_steps']-df_ck1['total_idle_steps']
df_ck1['Active_Steps_pct']=df_ck1['Active_Steps']/df_ck1['total_steps']*100

df_ck1=df_ck1.sort_values('Active_Steps_pct')

df1=df_ck1[['total_steps','Active_Steps_pct']]

df1=df1.sort_values('Active_Steps_pct')

plt.rcParams["figure.figsize"] = (3.1,2.6)
fig, axs = plt.subplots()
x=(np.arange(len(df1))/(len(df1)-1))*100
axs.plot(df1['Active_Steps_pct'],x,label="% Active Time",linestyle='--',linewidth=2,color='blue')
axs.set_xlabel('% Active Time',fontsize=18)
axs.set_ylabel('Empirical CDF \n(% of Jobs)',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylim(0,100)
plt.xlim(0,100)

axs.set_xticks([0,25,50,75,100])
axs.set_yticks([0,25,50,75,100])
plt.grid(color='darkgrey', linestyle=':')